# Fast haze removal implementation

In this notebook, we will implement the method described in the article "Fast Haze Removal for a Single Remote Sensing Image Using Dark Channel Prior".

## Import of libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import holidays
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt

### Import of images

### Algotihtm implementation

SyntaxError: invalid character in identifier (<ipython-input-3-e3b5186933bd>, line 1)